In [1]:
import pandas as pd
import numpy as np
import math
from scipy import stats

In [2]:
available_stocks = pd.read_json('curated_stocks_all.json')
stocks_10_to_30 = pd.read_json('stocks_10_to_30.json')
stocks_all = pd.read_json('stocks_all.json')

In [3]:
available_stocks_list = available_stocks['symbol'].tolist()

In [4]:
stocks_10_to_30 = stocks_10_to_30[stocks_10_to_30['Symbol'].isin(available_stocks_list)]
stocks_all = stocks_all[stocks_all['Symbol'].isin(available_stocks_list)]

In [5]:
stocks_10_to_30.drop(columns=['EBITDA', 'Gross_Profit'], inplace=True)
stocks_all.drop(columns=['EBITDA', 'Gross_Profit'], inplace=True)

In [6]:
stocks_all

,Symbol,Name,Price,P/E,P/B,P/S,Enterprise_Value,Volume,EV/EBITDA,EV/Gross_Profit
0,A,"Agilent Technologies, Inc.",121.770,26.837547,6.777320,5.334550,4.178070e+10,2376273,21.262443,11.334970
1,AA,Alcoa Corporation,36.190,NaN,1.261724,0.513273,8.274540e+09,6038960,13.953693,4.280673
11,AAIC,Arlington Asset Investment Corp,4.610,42.062044,0.704217,8.025374,6.710950e+08,184215,33.986377,62.572960
18,AAME,Atlantic American Corporation,1.830,NaN,0.365635,0.198635,9.022840e+07,510,NaN,NaN
19,AAN,"Aarons Holdings Company, Inc.",15.820,NaN,0.696580,0.217182,9.749530e+08,431964,1.214554,0.845988
...,...,...,...,...,...,...,...,...,...,...
7448,ZUO,"Zuora, Inc.",11.730,NaN,16.359376,3.892587,9.275390e+08,1366931,-13.532221,3.800595
7450,ZVIA,Zevia PBC,2.780,NaN,1.207918,0.740557,1.082520e+08,388615,-3.436244,1.576317
7454,ZYME,Zymeworks Inc.,7.460,2.890353,0.975850,1.180073,2.899130e+08,523906,1.568369,0.723917
7455,ZYNE,"Zynerba Pharmaceuticals, Inc.",0.365,NaN,0.378160,NaN,-2.329890e+07,104819,0.636433,106.250371


In [7]:
stocks_10_to_30 = stocks_10_to_30[(stocks_10_to_30['P/E'].isna() == False) &
               (stocks_10_to_30['P/B'].isna() == False) &
               (stocks_10_to_30['P/S'].isna() == False) &
               (stocks_10_to_30['EV/EBITDA'].isna() == False) &
               (stocks_10_to_30['EV/Gross_Profit'].isna() == False)]

In [8]:
stocks_all = stocks_all[(stocks_all['P/E'].isna() == False) &
               (stocks_all['P/B'].isna() == False) &
               (stocks_all['P/S'].isna() == False) &
               (stocks_all['EV/EBITDA'].isna() == False) &
               (stocks_all['EV/Gross_Profit'].isna() == False)]

In [9]:
stocks_10_to_30 = stocks_10_to_30.reset_index().drop(columns='index')
stocks_all = stocks_all.reset_index().drop(columns='index')

In [10]:
stocks_all

,Symbol,Name,Price,P/E,P/B,P/S,Enterprise_Value,Volume,EV/EBITDA,EV/Gross_Profit
0,A,"Agilent Technologies, Inc.",121.77,26.837547,6.777320,5.334550,4.178070e+10,2376273,21.262443,11.334970
1,AAIC,Arlington Asset Investment Corp,4.61,42.062044,0.704217,8.025374,6.710950e+08,184215,33.986377,62.572960
2,AAON,"AAON, Inc.",105.26,48.229095,10.029242,6.406767,5.429820e+09,477266,29.107605,23.657079
3,AAP,Advance Auto Parts Inc.,74.39,11.070764,1.646074,0.404917,1.157750e+10,1720160,12.572500,1.518573
4,AAPL,Apple Inc.,196.45,33.377508,61.810939,8.133349,2.666920e+12,38824111,21.544253,15.615931
...,...,...,...,...,...,...,...,...,...,...
1480,ZTO,ZTO Express (Cayman) Inc.,27.80,20.883413,2.178975,4.340474,2.344250e+10,1823985,15.283747,17.463247
1481,ZTS,Zoetis Inc.,188.09,42.503333,21.430362,10.949841,8.195800e+10,1500811,24.553026,14.966764
1482,ZUMZ,Zumiez Inc.,18.86,121.286174,0.902375,0.382324,4.266940e+08,447238,4.263572,1.397032
1483,ZYME,Zymeworks Inc.,7.46,2.890353,0.975850,1.180073,2.899130e+08,523906,1.568369,0.723917


In [11]:
# new_columns = ['PE Percentile','PS Percentile','PS Percentile', 'EV/EBITDA Percentile', 'EV/Gross_Profit Percentile']

# for col in new_columns:
#     stocks_10_to_30[col] = [np.NaN] * len(stocks_10_to_30)

In [12]:
stocks_10_to_30

,Symbol,Name,Price,P/E,P/B,P/S,Enterprise_Value,Volume,EV/EBITDA,EV/Gross_Profit
0,ABR,Arbor Realty Trust,15.0700,9.274518,1.158380,2.591772,1.586110e+10,3478155,16.544003,13.923004
1,ACI,"Albertsons Companies, Inc.",20.0800,8.734834,3.207632,0.131376,2.265800e+10,5447924,4.553731,1.180947
2,ACIW,"ACI Worldwide, Inc.",25.8700,20.607117,2.354861,2.088885,3.593990e+09,748280,10.426459,5.998461
3,ACRE,Ares Commercial Real Estate Corporation,11.5500,18.706352,0.819347,3.392792,2.160160e+09,584255,15.342808,13.998471
4,AGRO,Adecoagro S.A.,8.0500,5.536545,0.888362,0.892094,2.074830e+09,475535,12.818433,10.010663
...,...,...,...,...,...,...,...,...,...,...
212,WU,Western Union Company (The),12.8300,5.490330,10.052765,1.117435,6.737000e+09,5506871,6.201215,4.059167
213,XP,XP Inc.,11.4900,9.180093,2.485043,2.853403,1.942070e+10,5350907,23.185552,11.879771
214,ZEV,"Lightning eMotors, Inc",0.6722,1.135284,1.836370,1.770601,1.028380e+08,1264956,-1.554054,-17.159686
215,ZI,ZoomInfo Technologies Inc.,25.8500,158.237548,4.407425,9.103884,1.293470e+10,3889386,47.694322,14.569385


In [13]:
rv_dataframe = stocks_all
rv_dataframe = rv_dataframe.reset_index().drop(columns='index')
metrics = {
    'P/E':'PE Percentile',
    'P/B':'PB Percentile',
    'P/S':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/Gross_Profit':'EV/Gross_Profit Percentile'
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] \
            = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        
rv_dataframe

,Symbol,Name,Price,P/E,P/B,P/S,Enterprise_Value,Volume,EV/EBITDA,EV/Gross_Profit,PE Percentile,PB Percentile,PS Percentile,EV/EBITDA Percentile,EV/Gross_Profit Percentile
0,A,"Agilent Technologies, Inc.",121.77,26.837547,6.777320,5.334550,4.178070e+10,2376273,21.262443,11.334970,63.501684,83.501684,82.693603,84.915825,72.861953
1,AAIC,Arlington Asset Investment Corp,4.61,42.062044,0.704217,8.025374,6.710950e+08,184215,33.986377,62.572960,81.077441,5.185185,90.774411,93.804714,99.461279
2,AAON,"AAON, Inc.",105.26,48.229095,10.029242,6.406767,5.429820e+09,477266,29.107605,23.657079,84.511785,90.033670,87.609428,92.053872,93.737374
3,AAP,Advance Auto Parts Inc.,74.39,11.070764,1.646074,0.404917,1.157750e+10,1720160,12.572500,1.518573,22.760943,29.158249,9.427609,56.969697,5.319865
4,AAPL,Apple Inc.,196.45,33.377508,61.810939,8.133349,2.666920e+12,38824111,21.544253,15.615931,73.198653,98.922559,91.043771,85.252525,83.838384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,ZTO,ZTO Express (Cayman) Inc.,27.80,20.883413,2.178975,4.340474,2.344250e+10,1823985,15.283747,17.463247,50.707071,42.828283,78.114478,69.158249,87.003367
1481,ZTS,Zoetis Inc.,188.09,42.503333,21.430362,10.949841,8.195800e+10,1500811,24.553026,14.966764,81.414141,96.296296,95.218855,89.225589,82.289562
1482,ZUMZ,Zumiez Inc.,18.86,121.286174,0.902375,0.382324,4.266940e+08,447238,4.263572,1.397032,94.949495,8.282828,8.417508,11.043771,4.713805
1483,ZYME,Zymeworks Inc.,7.46,2.890353,0.975850,1.180073,2.899130e+08,523906,1.568369,0.723917,3.232323,9.898990,35.690236,2.693603,1.750842


In [14]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Symbol,Name,Price,P/E,P/B,P/S,Enterprise_Value,Volume,EV/EBITDA,EV/Gross_Profit,PE Percentile,PB Percentile,PS Percentile,EV/EBITDA Percentile,EV/Gross_Profit Percentile,RV Score
0,A,"Agilent Technologies, Inc.",121.77,26.837547,6.777320,5.334550,4.178070e+10,2376273,21.262443,11.334970,63.501684,83.501684,82.693603,84.915825,72.861953,77.494949
1,AAIC,Arlington Asset Investment Corp,4.61,42.062044,0.704217,8.025374,6.710950e+08,184215,33.986377,62.572960,81.077441,5.185185,90.774411,93.804714,99.461279,74.060606
2,AAON,"AAON, Inc.",105.26,48.229095,10.029242,6.406767,5.429820e+09,477266,29.107605,23.657079,84.511785,90.033670,87.609428,92.053872,93.737374,89.589226
3,AAP,Advance Auto Parts Inc.,74.39,11.070764,1.646074,0.404917,1.157750e+10,1720160,12.572500,1.518573,22.760943,29.158249,9.427609,56.969697,5.319865,24.727273
4,AAPL,Apple Inc.,196.45,33.377508,61.810939,8.133349,2.666920e+12,38824111,21.544253,15.615931,73.198653,98.922559,91.043771,85.252525,83.838384,86.451178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,ZTO,ZTO Express (Cayman) Inc.,27.80,20.883413,2.178975,4.340474,2.344250e+10,1823985,15.283747,17.463247,50.707071,42.828283,78.114478,69.158249,87.003367,65.562290
1481,ZTS,Zoetis Inc.,188.09,42.503333,21.430362,10.949841,8.195800e+10,1500811,24.553026,14.966764,81.414141,96.296296,95.218855,89.225589,82.289562,88.888889
1482,ZUMZ,Zumiez Inc.,18.86,121.286174,0.902375,0.382324,4.266940e+08,447238,4.263572,1.397032,94.949495,8.282828,8.417508,11.043771,4.713805,25.481481
1483,ZYME,Zymeworks Inc.,7.46,2.890353,0.975850,1.180073,2.899130e+08,523906,1.568369,0.723917,3.232323,9.898990,35.690236,2.693603,1.750842,10.653199


In [15]:
rv_dataframe.sort_values('RV Score', ascending=False, inplace=True)
rv_dataframe.reset_index(inplace=True, drop=True)

In [16]:
price_less_50 = rv_dataframe[rv_dataframe['Price'] <= 50]
volume_over_500k =  price_less_50[rv_dataframe['Volume'] >= 1000000]
volume_over_500k[:50]

<ipython-input-16-3a6a817665df>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  volume_over_500k =  price_less_50[rv_dataframe['Volume'] >= 1000000]


,Symbol,Name,Price,P/E,P/B,P/S,Enterprise_Value,Volume,EV/EBITDA,EV/Gross_Profit,PE Percentile,PB Percentile,PS Percentile,EV/EBITDA Percentile,EV/Gross_Profit Percentile,RV Score
32,DV,"DoubleVerify Holdings, Inc.",42.10,141.560188,7.942048,15.889698,4.973780e+09,1604282,43.369054,14.619133,95.892256,86.464646,98.720539,96.363636,81.346801,91.757576
39,DOCS,"Doximity, Inc.",35.73,67.542533,7.172538,18.197444,6.063220e+09,1150962,43.927464,16.586024,90.303030,84.646465,99.259259,96.430976,85.117845,91.151515
52,ROL,"Rollins, Inc.",40.83,51.540015,15.867032,7.457916,2.157960e+10,1948247,34.423594,16.649667,85.925926,94.949495,90.033670,94.276094,85.387205,90.114478
57,SHLS,"Shoals Technologies Group, Inc.",25.96,29.699119,12.286605,13.310396,3.581940e+09,1974444,38.608892,29.399202,67.878788,92.794613,97.037037,95.555556,96.161616,89.885522
86,PRVA,"Privia Health Group, Inc.",27.92,235.214827,6.415957,2.278103,3.175550e+09,1258017,807.616989,24.387725,97.508418,82.154882,58.855219,99.865320,94.141414,86.505051
91,ARRY,"Array Technologies, Inc.",19.05,296.267496,23.070775,1.742884,4.225780e+09,2272148,25.631915,32.978609,98.047138,96.565657,49.831650,89.562290,96.902357,86.181818
94,ZI,ZoomInfo Technologies Inc.,25.57,101.750895,4.547937,9.394121,1.068050e+10,9917931,35.249175,12.030300,93.872054,73.670034,92.996633,94.545455,74.410774,85.898990
103,HALO,"Halozyme Therapeutics, Inc.",42.96,32.911974,34.194837,9.150694,6.519240e+09,1175329,19.441267,13.648171,72.188552,97.912458,92.727273,82.222222,79.461279,84.902357
110,WPM,Wheaton Precious Metals Corp,44.83,33.338964,3.025631,19.863210,2.101094e+10,1121695,32.149322,38.871617,72.996633,57.845118,99.595960,93.400673,97.912458,84.350168
128,INVH,Invitation Homes Inc.,35.50,68.918657,2.109064,9.956086,2.829340e+10,4299062,21.637276,42.033843,90.505051,41.616162,94.343434,85.387205,98.249158,82.020202


In [17]:
# rv_dataframe.to_csv('10_to_30_vol_over_500k_result.csv', index=False)

In [18]:
'Talos', 'HOUS', 'TK'

('Talos', 'HOUS', 'TK')